In [ ]:
# %env autocnet_config=/home/tthatcher/demo.yml
import gdal

In [ ]:
!rm gis.db

In [ ]:
from autocnet_server.db import model
import json
import os
import sys
from ctypes.util import find_library
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, String, Integer, Float, JSON, ForeignKey, Boolean, LargeBinary, UniqueConstraint
# from sqlalchemy.dialects.postgresql import ARRAY, JSONB
from sqlalchemy.orm import relationship, backref
from sqlalchemy.sql import select, func
from geoalchemy2 import Geometry
from geoalchemy2.shape import to_shape
from sqlalchemy import Sequence
import json
import pandas as pd
from sqlalchemy.types import (
    Integer,
    String,
    TypeDecorator,
    Numeric,
    )

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Sequence
from sqlalchemy.event import listen

In [ ]:
Base = declarative_base()

# Custom class for Array
class ArrayType(TypeDecorator):
    """ Sqlite-like does not support arrays.
        Let's use a custom type decorator.

        See http://docs.sqlalchemy.org/en/latest/core/types.html#sqlalchemy.types.TypeDecorator
    """
    
    impl = String

    def process_bind_param(self, value, dialect):
        return json.dumps(value)

    def process_result_value(self, value, dialect):
        return json.loads(value)

    def copy(self):
        return ArrayType(self.impl.length)

# Custom class for JSON
class Json(TypeDecorator):

    @property
    def python_type(self):
        return object

    impl = String

    def process_bind_param(self, value, dialect):
        return json.dumps(value)

    def process_literal_param(self, value, dialect):
        return value

    def process_result_value(self, value, dialect):
        try:
            return json.loads(value)
        except (ValueError, TypeError):
            return None

class Cameras(Base):
    __tablename__ = 'cameras'
    id = Column(Integer, primary_key=True, autoincrement=True)
    image_id = Column(Integer, ForeignKey("edges.id", ondelete="CASCADE"), unique=True)
    camera = Column(Json())
    
class Element(Base):
    __tablename__ = 'elements'

    id = Column(Integer(),
                Sequence('element_id_seq'),
                primary_key = True)
    myarray = Column(ArrayType())
    mycolumn = Column(Json())
    
class Edges(Base):
    __tablename__ = 'edges'
    id = Column(Integer, primary_key=True, autoincrement=True)
    source = Column(Integer)
    destination = Column(Integer)
    ring = Column(ArrayType())
    fundamental = Column(ArrayType())
    active = Column(Boolean)
    masks = Column(Json())
    
class Overlay(Base):
    __tablename__ = 'overlay'
    id = Column(Integer, primary_key=True, autoincrement=True)
    overlaps = Column(ArrayType())
    geom = Column(Geometry(geometry_type='POLYGON', management=True, use_st_prefix=False))

class Lake(Base):
    __tablename__ = 'lake'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    geom = Column(Geometry(geometry_type='POLYGON', management=True, use_st_prefix=False))

def load_spatialite(dbapi_conn, connection_record):
    spatial_path = find_library('spatialite')
    dbapi_conn.enable_load_extension(True)
    dbapi_conn.load_extension(spatial_path)

In [ ]:
engine = create_engine('sqlite:///gis.db', echo=True)

listen(engine, 'connect', load_spatialite)

conn = engine.connect()

conn.execute(select([func.InitSpatialMetaData()]))

conn.close()

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)

session = Session()

In [ ]:

# Creates a df out of dict and dumps to json
data = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data)
df_json = df.to_json(orient='index')

# # creates a test of an array and json
lake = Lake(name='Majeur', geom='POLYGON((0 0,1 0,1 1,0 1,0 0))')
test_data = Element(id=69, myarray =[1, 2, 3, 4, 5], mycolumn=df_json)
gm = Overlay(id=75, overlaps=[1,2,3,4,5], geom='POLYGON((0 0,1 0,1 1,0 1,0 0))')
# # Adds that test
session.add(test_data)
session.add(gm)
session.add(lake)

session.commit()


# # Query the db for the test_data values
our_user = session.query(Element).filter_by(id=69).first()
our_geom = session.query(Overlay).filter_by(id=75).first()

# # Output both values
# our_user.mycolumn
our_geom.geom
# lake.geom
# our_geom.overlaps

# from sqlalchemy import inspect
# inspector = inspect(engine)

# for table_name in inspector.get_table_names():
#     for column in inspector.get_columns(table_name):
#         print("Column: %s" % column['name'])
# our_geom.geom